In [1]:
import pandas as pd
import sqlite3 as sql

In [2]:
conn = sql.connect("G:\My Drive\Prepinsta - Data Analytics\Week 5\Week 5 - Task 5\sql-murder-mystery.db")

In [4]:
conn

## 1. Retrieve Crime Scene Report:
### Task: Run a query to retrieve the crime scene report for the murder that occurred on Jan.15, 2018, in SQL City. Gather all available details from the report.

In [6]:
query1 = """
SELECT *FROM crime_scene_report 
WHERE date = 20180115 
AND type = "murder" 
AND city = "SQL City";
"""

pd.read_sql_query(query1,conn)

,date,type,description,city
0,20180115,murder,Security footage shows that there were 2 witne...,SQL City


## 2. Witness Personal Details:
### Task: Check the personal details of witnesses involved in the case. Retrieve their names, addresses, and any other relevant information.

#### 1st Witness -->

In [7]:
query2 = """
SELECT *FROM person 
WHERE address_street_name = "Northwestern Dr" 
ORDER BY address_number 
DESC LIMIT 1;
"""

pd.read_sql_query(query2,conn)

,id,name,license_id,address_number,address_street_name,ssn
0,14887,Morty Schapiro,118009,4919,Northwestern Dr,111564949


#### 2nd Witness -->

In [8]:
query3 = """
SELECT *FROM person 
WHERE address_street_name = "Franklin Ave" 
AND name LIKE "Annabel%"; 
"""

pd.read_sql_query(query3,conn)

,id,name,license_id,address_number,address_street_name,ssn
0,16371,Annabel Miller,490173,103,Franklin Ave,318771143


## 3. View Witness Interviews:
### Task: Access the recorded interviews of witnesses conducted after the murder. Gather insights into their statements and potential clues.

#### 1st Witness's transcript -->

In [9]:
query4 = """
SELECT transcript 
FROM interview 
WHERE person_id  = 16371;
"""

pd.read_sql_query(query4,conn)

,transcript
0,"I saw the murder happen, and I recognized the ..."


#### 2nd Witness transcript -->

In [10]:
query5 = """
SELECT transcript 
FROM interview 
WHERE person_id  = 14887;
"""

pd.read_sql_query(query5,conn)

,transcript
0,I heard a gunshot and then saw a man run out. ...


## 4. Check Gym Database:
### Task: Investigate the gym database using details obtained from the crime scene report and witness interviews. Look for any gym-related information that might be relevant.

#### Cross referencing 1st witness transcript -->

In [11]:
query6 = """
SELECT membership_id 
FROM get_fit_now_check_in 
WHERE check_in_date = 20180109;
"""

pd.read_sql_query(query6,conn)

,membership_id
0,X0643
1,UK1F2
2,XTE42
3,1AE2H
4,6LSTG
5,7MWHJ
6,GE5Q8
7,48Z7A
8,48Z55
9,90081


#### Cross referencing 2nd witness transcript -->

In [12]:
query7 = """
SELECT *FROM get_fit_now_member 
WHERE id LIKE "48Z%" 
AND membership_status = "gold";
"""

pd.read_sql_query(query7,conn)

,id,person_id,name,membership_start_date,membership_status
0,48Z7A,28819,Joe Germuska,20160305,gold
1,48Z55,67318,Jeremy Bowers,20160101,gold


## 5. Check Car Details:
### Task: Examine the car details associated with the crime scene. Retrieve information about the vehicles present during the incident.

In [13]:
query8 = """
SELECT id FROM drivers_license 
WHERE plate_number LIKE "%H42W%";
"""

pd.read_sql_query(query8,conn)

,id
0,183779
1,423327
2,664760


## 6. Prsonal Details:
### Task: Identify and collect personal details mentioned in the previous query. This includes names, addresses, and any additional details.

In [14]:
query9 = """ 
SELECT *FROM person 
WHERE license_id IN (183779, 423327, 664760);
"""

pd.read_sql_query(query9,conn)

,id,name,license_id,address_number,address_street_name,ssn
0,51739,Tushar Chandra,664760,312,Phi St,137882671
1,67318,Jeremy Bowers,423327,530,"Washington Pl, Apt 3A",871539279
2,78193,Maxine Whitely,183779,110,Fisk Rd,137882671


## 7. Membership Status at the Gym:
### Task: Determine who is identified in the previous query as a member of the gym. Utilize the gym database to confirm their membership status.

In [15]:
query10 = """     
SELECT *FROM person 
WHERE license_id IN (183779, 423327, 664760) 
AND id IN (28819,67318);
"""

pd.read_sql_query(query10,conn)

,id,name,license_id,address_number,address_street_name,ssn
0,67318,Jeremy Bowers,423327,530,"Washington Pl, Apt 3A",871539279


#### Congratulations !! We might have found the murderer but first we should read what he said in the witness interview. I mean does he already accepted the murder charges or not.

In [16]:
query11 = """     
SELECT transcript 
FROM interview 
WHERE person_id  = 67318;
"""

pd.read_sql_query(query11,conn)

,transcript
0,I was hired by a woman with a lot of money. I ...


#### Here we can see that he is not the actual murderer, he was hired by some woman. He also provided some insights about the woman. We need to investigate this information about the woman to catch the actual killer.

##### 1st insight : She's around 5'5" (65") or 5'7" (67"). She has red hair and she drives a Tesla Model S.

In [17]:
query12 = """     
SELECT *FROM drivers_license 
WHERE gender  = "female" 
AND height BETWEEN 65 AND 67 
AND hair_color = "red" 
AND car_make = "Tesla" 
AND car_model  = "Model S";
"""

pd.read_sql_query(query12,conn)

,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model
0,202298,68,66,green,red,female,500123,Tesla,Model S
1,291182,65,66,blue,red,female,08CM64,Tesla,Model S
2,918773,48,65,black,red,female,917UU3,Tesla,Model S


##### To establish a link between 1st insight and 2nd insight, we have to find the person's unique identity number from there license id.

In [18]:
query13 = """
SELECT id FROM person 
WHERE license_id IN (202298,291182,918773);
"""

pd.read_sql_query(query13,conn)

,id
0,78881
1,90700
2,99716


##### 2nd insight : She attended the SQL Symphony Concert 3 times in December 2017.

In [19]:
query14 = """ 
SELECT *FROM facebook_event_checkin 
WHERE person_id IN (78881,90700,99716) 
AND event_name  = "SQL Symphony Concert" 
AND date LIKE "201712%";
"""

pd.read_sql_query(query14,conn)

,person_id,event_id,event_name,date
0,99716,1143,SQL Symphony Concert,20171206
1,99716,1143,SQL Symphony Concert,20171212
2,99716,1143,SQL Symphony Concert,20171229


#### We have found the murderer. Let's find his personal detail to know more about her.

In [20]:
query15 = """     
SELECT *FROM person 
WHERE id  = 99716;
"""

pd.read_sql_query(query15,conn)

,id,name,license_id,address_number,address_street_name,ssn
0,99716,Miranda Priestly,202298,1883,Golden Ave,987756388


#### Insert the name of the murderer in the solution Table.

In [36]:
conn.execute("""INSERT INTO solution VALUES ("Miranda Priestly",1);""")

In [37]:
query17 = """     
SELECT *FROM solution;
"""

pd.read_sql_query(query17,conn)

,user,value
0,Miranda Priestly,1
